In [1]:
import numpy as np
import pandas as pd

In [2]:
a1 = pd.read_csv('Datasets/case_study1.csv')
a2 = pd.read_csv('Datasets/case_study2.csv')

In [3]:
df1 = a1.copy()

In [4]:
df2 = a2.copy()

In [6]:
df1 = df1.replace(-99999, np.nan)

In [7]:
df2 = df2.replace(-99999, np.nan)

In [8]:
# nan in df1 is too less, we directly remove the rows from it.
df1 = df1.dropna()

In [9]:
nan_percent = (df2.isna().sum() / df2.shape[0]) * 100
nan_cols = nan_percent[nan_percent > 10].index

In [10]:
df2 = df2.drop(columns=nan_cols)

In [11]:
df2 = df2.dropna()

In [12]:
df = pd.merge(df1, df2, on="PROSPECTID", how="inner")

In [13]:
df = df.drop(columns=['PROSPECTID'])

In [14]:
y = df['Approved_Flag']
X = df.drop(columns=['Approved_Flag'])

In [15]:
object_cols = X.select_dtypes(include=['object']).columns.tolist()

In [16]:
nominal_cols = ['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [17]:
ordinal_cols = ['EDUCATION']
order = {'OTHERS':1, 'SSC':2, '12TH':3, 'UNDER GRADUATE':4, 'GRADUATE':5, 'POST-GRADUATE':6, 'PROFESSIONAL':7}

In [18]:
X['EDUCATION'] = X['EDUCATION'].replace(order)

/tmp/ipykernel_373021/798088432.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['EDUCATION'] = X['EDUCATION'].replace(order)


In [19]:
X_encoded = pd.get_dummies(X, columns=nominal_cols, drop_first=True)

In [20]:
classes = y.unique()

In [21]:
classes.sort()

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=42, test_size=0.2)

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [25]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [26]:
model = XGBClassifier(
                        objective='multi:softmax', 
                        num_class=4,
                        colsample_bytree=0.3,
                        learning_rate=0.1,
                        max_depth=5,
                        alpha=1,
                        n_estimators=100
                    )

In [27]:
model.fit(X_train, y_train)

XGBClassifier(alpha=1, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=4, ...)

In [30]:
y_pred_test = model.predict(X_test)

In [31]:
accuracy_score(y_test, y_pred_test)

0.9960621541081311

In [32]:
y_pred_train = model.predict(X_train)

In [33]:
accuracy_score(y_train, y_pred_train)

0.996753765100314

In [34]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)
for i,v in enumerate(classes):
    print(f"Class: {v}")
    print(f"Precision: {precision[i]}")
    print(f"recall: {recall[i]}")
    print(f"f1_score: {f1_score[i]}")
    print()

Class: P1
Precision: 0.9952426260704091
recall: 0.9703153988868275
f1_score: 0.9826209488022546

Class: P2
Precision: 1.0
recall: 1.0
f1_score: 1.0

Class: P3
Precision: 0.9769949676491733
recall: 0.9963343108504399
f1_score: 0.9865698729582577

Class: P4
Precision: 1.0
recall: 1.0
f1_score: 1.0



In [35]:
X_encoded.shape

(46978, 74)